In [6]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.translate.ribes_score import word_rank_alignment
from numpy.lib.shape_base import split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.layers import LSTM,Dense,Dropout,Input,Embedding,Activation,Flatten
from keras.models import Model
from tensorflow.keras.models import Sequential

In [7]:
data = pd.read_csv("DATA/spam.csv",encoding = "ISO-8859-1")

In [8]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [9]:
data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis = 1,inplace = True)
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# **TEXT PROCESSING**

In [10]:
nltk.download('stopwords')
nltk.download('all')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dinek\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\dinek\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\dinek\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\dinek\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\dinek\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basqu

True

In [11]:
porterStemmer = PorterStemmer()
input = []

In [12]:
for i in range(0,5572):
  v2 = data['v2'][i]

  #removing punctuation
  v2 = re.sub('[^a-zA-Z]',' ',v2)

  #converting to lower case
  v2 = v2.lower()

  #splitting the sentence
  v2 = v2.split()

  #removing the stopwords and stemming
  v2 = [porterStemmer.stem(word) for word in v2 if not word in set(stopwords.words('english'))]

  v2 = ' '.join(v2)

  input.append(v2)

In [13]:
#creating document term matrix
countVectorizer = CountVectorizer(max_features=2000)

In [14]:
x = countVectorizer.fit_transform(input).toarray()

In [15]:
le = preprocessing.LabelEncoder()

data['v1'] = le.fit_transform(data['v1'])
data['v1'].unique()

array([0, 1])

In [16]:
y = data['v1'].values

In [17]:
y = y.reshape(-1,1)

In [18]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.4)

# **Model -ANN**

In [19]:
model = Sequential()

In [20]:
model.add(Dense(1565,activation = "relu"))
model.add(Dense(3000,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))
model.add(Flatten())

In [21]:
model.compile(optimizer = "adam",loss = "binary_crossentropy", metrics = ["accuracy"])

In [22]:
model.fit(x_train,y_train,epochs = 15)

Epoch 1/15
105/105 [==============================] - 8s 65ms/step - loss: 0.1246 - accuracy: 0.9641
Epoch 2/15
105/105 [==============================] - 7s 69ms/step - loss: 0.0108 - accuracy: 0.9967
Epoch 3/15
105/105 [==============================] - 7s 67ms/step - loss: 0.0032 - accuracy: 0.9991
Epoch 4/15
105/105 [==============================] - 7s 66ms/step - loss: 0.0021 - accuracy: 0.9997
Epoch 5/15
105/105 [==============================] - 7s 68ms/step - loss: 0.0019 - accuracy: 0.9997
Epoch 6/15
105/105 [==============================] - 7s 65ms/step - loss: 0.0018 - accuracy: 0.9997
Epoch 7/15
105/105 [==============================] - 7s 65ms/step - loss: 0.0018 - accuracy: 0.9997
Epoch 8/15
105/105 [==============================] - 7s 66ms/step - loss: 0.0017 - accuracy: 0.9997
Epoch 9/15
105/105 [==============================] - 7s 66ms/step - loss: 0.0015 - accuracy: 0.9997
Epoch 10/15
105/105 [==============================] - 7s 71ms/step - loss: 0.0018 - accura

In [23]:
model.save("SMS_Spam_Classifier.h5")

In [24]:
ham = "im donee. come pick me up"
spam = "WINNER$$$$ SMS REPLY 'WIN'"
message = re.sub('[^a-zA-Z]',' ',spam)

In [25]:
message = message.split()
message = [porterStemmer.stem(word) for word in message if not word in set(stopwords.words('english')) ]
message = ' '.join(message)

In [26]:
message

'winner sm repli win'

In [27]:
countVectorizer.transform([message])

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [28]:
message1 = countVectorizer.transform([message])
message1

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [29]:
import numpy as np
pred = model.predict(message1.astype(float))

1/1 [==============================] - 0s 148ms/step


In [30]:
msg = re.sub('[^a-zA-Z]',' ',ham)

In [31]:
msg = msg.split()
msg = [porterStemmer.stem(word) for word in msg if not word in set(stopwords.words('english'))]
msg = ' '.join(msg)

In [32]:
countVectorizer.transform([msg])

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [33]:
pred1 = model.predict(countVectorizer.transform([msg]))

1/1 [==============================] - 0s 99ms/step


In [34]:
pred1 > 0.5

array([[False]])